这个是用来解决三位思维数据等高维度的问题。 实践中通过层级索引hierarchical indexing 配合多个不同的等级 可以将高位转为一维或者二维的低维度数据

In [1]:
import numpy as np
import pandas as pd

### 3.6.1 多级索引Series

In [2]:
#用一个元组表示索引
index = [('California', 2000), ('California', 2001), ('New York', 2000), ('New York', 2001)]
populaiton = [165416,213884,665468,598949]
pop= pd.Series(populaiton, index= index)

In [3]:
pop

(California, 2000)    165416
(California, 2001)    213884
(New York, 2000)      665468
(New York, 2001)      598949
dtype: int64

**好方法!!!** Pandas 多级索引

In [4]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York'], [2000, 2001]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [5]:
pop = pop.reindex(index)
pop

California  2000    165416
            2001    213884
New York    2000    665468
            2001    598949
dtype: int64

In [6]:
pop[:, 2001]

California    213884
New York      598949
dtype: int64

3.高维数据的多级索引

In [7]:
#unstack()方法可以快速将一个多级索引的Series转化为普通的DataFrame

In [8]:
pop_df = pop.unstack()

In [9]:
pop_df

,2000,2001
California,165416,213884
New York,665468,598949


In [10]:
pop_df.stack()

California  2000    165416
            2001    213884
New York    2000    665468
            2001    598949
dtype: int64

### 3.6.2 多级索引的创建

In [11]:
df = pd.DataFrame(np.random.rand(4,2), index = [['a','a', 'b', 'b'],  [1,2,1,2]], columns= [ 'data1', 'data2'])

In [12]:
df

data1     data2
a 1  0.860844  0.944118
  2  0.403881  0.358596
b 1  0.988218  0.806094
  2  0.461608  0.547859

In [13]:
#为Series或者DataFrame 创建多级索引的最直接方法是将index 参数设置为至少二维的索引数据 MultiIndex 创建在后台完成


1.显式创建多级索引

In [14]:
pd.MultiIndex.from_arrays([['a','a','b','b'], [1,2,1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [15]:
#可以用两个索引的笛卡尔积创建MultiIndex：
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

2.多级索引的等级名称

In [16]:
pop

California  2000    165416
            2001    213884
New York    2000    665468
            2001    598949
dtype: int64

In [17]:
pop.index.names = ['state', 'year']

In [18]:
pop

state       year
California  2000    165416
            2001    213884
New York    2000    665468
            2001    598949
dtype: int64

3.多级列索引

可以在列索引的第一级查询从而检索出整个DataFrame

### 3.6.3多级索引的取值与切片

In [19]:
pop

state       year
California  2000    165416
            2001    213884
New York    2000    665468
            2001    598949
dtype: int64

通过对多个级别的索引值获取单个元素

In [21]:
pop['California', 2000]#相当于一个二维坐标定位

165416

In [22]:
# 局部取值 只索引某一个层级 
pop['California'] 

year
2000    165416
2001    213884
dtype: int64

In [25]:
#局部切片
pop.loc['California':'New York']

state       year
California  2000    165416
            2001    213884
New York    2000    665468
            2001    598949
dtype: int64

In [26]:
#剩下的和基本的索引操作是一样的

2. DataFrame多级索引

In [27]:
#先创建一个体检数据

In [28]:
index = pd.MultiIndex.from_product([[2013, 2014], [1,2]], names =['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names = ['subject', 'type'])
data = np.round(np.random.rand(4,6) , 1)
data[:,::2] *= 10
data += 37

In [29]:
health_data = pd.DataFrame(data, index = index ,columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  37.3  45.0  37.2  45.0  37.4
     2      47.0  37.3  39.0  37.2  41.0  37.7
2014 1      42.0  37.7  42.0  37.4  37.0  37.3
     2      47.0  37.0  41.0  37.8  46.0  37.3

In [30]:
#选取Guido的心率数据
health_data['Guido',"HR"]

year  visit
2013  1        45.0
      2        39.0
2014  1        42.0
      2        41.0
Name: (Guido, HR), dtype: float64

In [31]:
#也可以好用loc iloc ix索引器
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      41.0  37.3
     2      47.0  37.3

In [32]:
health_data.loc[:, ('Bob','HR')]

year  visit
2013  1        41.0
      2        47.0
2014  1        42.0
      2        47.0
Name: (Bob, HR), dtype: float64

### 3.6.4 多级索引行列转换


1.有序的索引和无序索引 。。。。xian